In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from junno import log
log

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
from sources import load_dataset, default_config
import junno.datasets as D
import numpy as np


In [4]:
cfg = default_config()
cfg['script-arguments']['gpus']='0'
cfg['model']['steered']='all'
cfg['training']['dataset-file'] =  'av.h5'
cfg['data-augmentation']['rotation'] = True
cfg['data-augmentation']['elastic'] = False
train, val, test = load_dataset(cfg)

In [5]:
data = {'x':[], 'y':[], 'mask': [], 'angle': [], 'angle_xy': [], 'vec_norm': []}
for i, d in enumerate(train):
    for k in data.keys():
        data[k].append(d[k].numpy())
    if i>=4:
        break
data = {k: np.concatenate(v) for k,v in data.items()}


In [6]:
def angle2field(angle):
    return np.stack([np.cos(angle), np.sin(angle)])
def norm(vec_norm):
    return vec_norm / np.linalg.norm(vec_norm, axis=0, keepdims=True)
def diff(angle_xy, angle_xy_true):
    return np.abs(angle_xy-angle_xy_true)
dataset = (D.from_numpy(**data).apply('raw', lambda x: x[3:])
            .apply('vec_norm', norm)
            .apply('angle_xy_true', angle2field, format='Field')
            .apply('diff', diff).format('vec_norm', 'Field').format('angle_xy', 'Field'))

dataset.col['angle_xy_true'].format.background = 'raw'
dataset.col['angle_xy'].format.background = 'raw'
dataset.col['vec_norm'].format.background = 'raw'
dataset.col

AttributeDict([('diff', diff: (2, 565, 565) float32),
               ('angle_xy_true', angle_xy_true: (2, 565, 565) float32),
               ('vec_norm', vec_norm: (2, 565, 565) float32),
               ('raw', raw: (3, 565, 565) float32),
               ('x', x: (6, 565, 565) float32),
               ('y', y: (565, 565) float32),
               ('mask', mask: (565, 565) float32),
               ('angle', angle: (565, 565) float32),
               ('angle_xy', angle_xy: (2, 565, 565) float32)])

In [7]:
#dataset.reshape('angle_xy,raw,vec_norm,mask,y', 128).at[:2, 'angle_xy,raw,vec_norm,mask,y']

In [8]:
np.abs(dataset[:, 'diff'][...,dataset[:2, 'mask']!=0]).mean()

4.625135e-05